<a href="https://colab.research.google.com/github/MatheusSilva020/Jogo-penaltis/blob/main/Dados_Calcio_Lega_Serie_A_19_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Bibliotecas:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


In [2]:
#obter_dados_partida(): recebe uma tag geral, procura e retorna os dados da partida.
def obter_dados_partida(dadosJogo):
  span = dadosJogo.find('span')
  a = dadosJogo.find(class_='note')

  #criando data e hora da partida:
  #removendo as tags:
  data = span.get_text().strip().replace('<span>', '').replace('</span>','')
  #dia - inicio da span:
  dia = data[0:10]
  #hora - final da span
  hora = data
  hora = hora.replace(hora[0:11], '')

  #criando os registros da partida(estadio e árbitro):
  span = str(span)
  a = str(a)
  reg_string = str(dadosJogo)
  reg_string = reg_string.replace(span, '').replace(a, '')
  #removendo as tags:
  reg_string = reg_string.replace(reg_string[reg_string.find('<div'):reg_string.find('Stadium: ')], '')
  reg_string = reg_string.replace(reg_string[reg_string.find('Live: '):], '')
  #obtendo o estadio:
  estadio = reg_string[reg_string.find(': ') + 2: reg_string.find('<br/>')]
  #obtendo o arbitro:
  reg_string = reg_string.replace(reg_string[reg_string.find('Stadium: '):reg_string.find('Referee: ')], '')
  arbitro = reg_string[reg_string.find(': ') + 2: reg_string.find('<br/>')]

  return dia, hora, estadio, arbitro


In [3]:
#obter_resultado_partida_casa(): recebe a tag, obtem e retorna os dados do time mandante.
def obter_resultado_partida_casa(site):
  #obtendo time casa:
  golsCasa = site.find('span')
  timeCasa = site.find(class_='nomesquadra')
  golsCasa = golsCasa.get_text()
  timeCasa = timeCasa.get_text()

  return timeCasa, golsCasa

In [4]:
#obter_resultado_partida_fora(): recebe a tag, obtem e retorna os dados do time visitante.
def obter_resultado_partida_fora(site):
  #obtendo time fora:
  golsFora = site.find('span')
  timeFora = site.find(class_='nomesquadra')
  golsFora = golsFora.get_text()
  timeFora = timeFora.get_text()

  return golsFora, timeFora

In [5]:
#adicionar_matriz: recebe a matriz final e as listas de dados de cada rodada, preenchendo a matriz final.
def adicionar_matriz(matrizFinal, dados, dadosCasa, dadosFora):
  #loop para preencher todos os resultados(cada rodada de uma vez):
  for i in range(10):
    line = []
    for j in range(4):
      line.append(dados[i][j])
    for j in range(2):
      line.append(dadosCasa[i][j])
    for j in range(2):
      line.append(dadosFora[i][j])
    matrizFinal.append(line)

In [6]:
#Lista para armazenar as séries que serão pesquisadas
calcioA = []
#Variável para armazenar o início do link que será utilizado para extrair os dados.
linkRaiz = 'https://www.legaseriea.it/en/serie-a/archive/2019-20/UNICO/UNI/'

#loop para obter todas as rodadas do campeonato(38):
for i in range(38):
  #obtendo o link de cada rodada:
  linkRodada = linkRaiz + str(i+1)
  requisition = requests.get(linkRodada)
  #transformando com beautifulsoup
  site = BeautifulSoup(requisition.content, 'html.parser')
  #organizando com prettiffy
  site.prettify()

  #obtendo os dados da partida(10 jogos por página):
  dadoJogo = site.find(class_='datipartita')
  dadosJogo = site.find_all(class_='datipartita')   
  dados = []            
  for dadoJogo in dadosJogo:                        
    dados.append(obter_dados_partida(dadoJogo))     

  #obtendo o resultado da partida:(10 jogos por página):
  #time casa:
  dadosCasa = []
  resultadoPartidaCasa = site.find(class_='col-xs-6 risultatosx')
  resultadosCasa = site.find_all(class_='col-xs-6 risultatosx')
  for resultadoPartidaCasa in resultadosCasa:
    dadosCasa.append(obter_resultado_partida_casa(resultadoPartidaCasa))
  #time fora:
  dadosFora = []
  resultadoPartidaFora = site.find(class_='col-xs-6 risultatodx')
  resultadosFora = site.find_all(class_='col-xs-6 risultatodx')
  for resultadoPartidaFora in resultadosFora:
    dadosFora.append(obter_resultado_partida_fora(resultadoPartidaFora))

  #adicionando os dados na matriz final:
  adicionar_matriz(calcioA, dados, dadosCasa, dadosFora)

In [7]:
#criando um dataframe com os dados:
df = pd.DataFrame(calcioA, columns = ['Data', 'Horário', 'Estádio', 'Árbitro', 'Casa', 'GolsCasa', 'GolsVisitante', 'Visitante'])
df

,Data,Horário,Estádio,Árbitro,Casa,GolsCasa,GolsVisitante,Visitante
0,24/08/2019,18:00,ENNIO TARDINI (Parma),FABIO MARESCA,Parma,0,1,Juventus
1,24/08/2019,20:45,ARTEMIO FRANCHI (Firenze),DAVIDE MASSA,Fiorentina,3,4,Napoli
2,25/08/2019,18:00,DACIA ARENA - STADIO FRIULI (Udine),FABRIZIO PASQUA,Udinese,1,0,Milan
3,25/08/2019,20:45,UNIPOL DOMUS (Cagliari),EUGENIO ABBATTISTA,Cagliari,0,1,Brescia
4,25/08/2019,20:45,MARCANTONIO BENTEGODI (Verona),ANTONIO GIUA,Hellas Verona,1,1,Bologna
...,...,...,...,...,...,...,...,...
375,02/08/2020,18:00,PAOLO MAZZA (Ferrara),ALESSANDRO PRONTERA,Spal,1,3,Fiorentina
376,02/08/2020,20:45,RENATO DALL'ARA (Bologna),ANTONIO DI MARTINO,Bologna,1,1,Torino
377,02/08/2020,20:45,LUIGI FERRARIS (Genova),MASSIMILIANO IRRATI,Genoa,3,0,Hellas Verona
378,02/08/2020,20:45,ETTORE GIARDINIERO - VIA DEL MARE (Lecce),MAURIZIO MARIANI,Lecce,3,4,Parma


In [8]:
#salvando o dataframe criado:
df.to_excel("LegaCalcioSerieA_2019_2020.xlsx", index = False)